In [1]:
import os
os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [2]:
##CREATE URM,ICM, ICM STACKED COMPLETELY TO TRAIN THE SINGLE RECOMMENDERS, AND STACKED PARTIALLY TO TUNE THE HYBRID
URM = create_URM()
ICM = create_ICM()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
combined_matrices = combine_matrices(ICM=ICM, URM=URM_train)

EvaluatorHoldout: Ignoring 13643 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13629 ( 0.2%) Users that have less than 1 test interactions


In [3]:
ICM_combined = combine_matrices(ICM=ICM,URM=URM)

In [4]:
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
### UserCF Recommender
UserCF_recommender = UserKNNCFRecommender(URM_train = ICM_combined)
UserCF_recommender.fit(topK= 450, shrink= 5.0, similarity= "tanimoto")

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 13984 (100.0%), 875.10 column/sec. Elapsed time 15.98 sec


In [ ]:
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=ICM_combined)
SLIME_recommender.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)

 14%|█████▎                                | 2504/18059 [01:38<04:22, 59.23it/s]

In [ ]:
similarity_userCF=UserCF_recommender.W_sparse
similarity_userCF

In [7]:
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
##HYPERPARAMETER TUNING OF HYBRID
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real
hyperparameters_range_dictionary = {
    "alpha": Real(0.0, 1.0)
}

recommender =[SLIME_recommender, UserCF_recommender]
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[combined_matrices, SLIME_recommender,UserCF_recommender],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class= ItemKNNSimilarityHybridRecommender,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 200
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root= ItemKNNSimilarityHybridRecommender.RECOMMENDER_NAME,
                    save_model="best"
                    )




Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 0.3879930325407157, 'beta': 0.5300963713206329}
EvaluatorHoldout: Processed 13632 (100.0%) in 19.09 sec. Users per second: 714
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 0.3879930325407157, 'beta': 0.5300963713206329} - results: PRECISION: 0.2648621, PRECISION_RECALL_MIN_DEN: 0.2718931, RECALL: 0.0829791, MAP: 0.1380037, MAP_MIN_DEN: 0.1416059, MRR: 0.5216325, NDCG: 0.2815365, F1: 0.1263681, HIT_RATE: 0.9426350, ARHR_ALL_HITS: 0.8450008, NOVELTY: 0.0054431, AVERAGE_POPULARITY: 0.6017848, DIVERSITY_MEAN_INTER_LIST: 0.8436923, DIVERSITY_HERFINDAHL: 0.9843630, COVERAGE_ITEM: 0.0792956, COVERAGE_ITEM_CORRECT: 0.0436901, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.9413919, DIVERSITY_GINI: 0.0066318, SHANNON_ENTROPY: 7.0241785, RATIO_DIVERSITY_HERFINDAHL: 0.9847330, RATIO_DIVERSITY_GINI: 0.0255898, RATIO_SHANNON_ENTROPY: 0.5644876, RATIO_AVERAGE_POPULARITY:

100%|████████████████████████████████████▉| 18056/18059 [17:05<00:00, 17.61it/s]


EvaluatorHoldout: Processed 13632 (100.0%) in 19.44 sec. Users per second: 701
SearchBayesianSkopt: Config 1 is suboptimal. Config: {'alpha': 0.9199502768022629, 'beta': 0.7734632935457475} - results: PRECISION: 0.2649648, PRECISION_RECALL_MIN_DEN: 0.2717060, RECALL: 0.0824949, MAP: 0.1377840, MAP_MIN_DEN: 0.1412615, MRR: 0.5197118, NDCG: 0.2810379, F1: 0.1258175, HIT_RATE: 0.9417547, ARHR_ALL_HITS: 0.8427966, NOVELTY: 0.0054438, AVERAGE_POPULARITY: 0.6013203, DIVERSITY_MEAN_INTER_LIST: 0.8441481, DIVERSITY_HERFINDAHL: 0.9844086, COVERAGE_ITEM: 0.0801816, COVERAGE_ITEM_CORRECT: 0.0438009, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.9405128, DIVERSITY_GINI: 0.0067122, SHANNON_ENTROPY: 7.0358285, RATIO_DIVERSITY_HERFINDAHL: 0.9847786, RATIO_DIVERSITY_GINI: 0.0259002, RATIO_SHANNON_ENTROPY: 0.5654238, RATIO_AVERAGE_POPULARITY: 3.0331547, RATIO_NOVELTY: 0.0284781, 

Iteration No: 2 ended. Evaluation done at random point.
Time taken: 19.5719
Function value obtained: -0.1378
Current m

SearchBayesianSkopt: Config 7 is suboptimal. Config: {'alpha': 0.8375792962488222, 'beta': 0.17355333873762538} - results: PRECISION: 0.2638644, PRECISION_RECALL_MIN_DEN: 0.2695546, RECALL: 0.0799896, MAP: 0.1359954, MAP_MIN_DEN: 0.1387482, MRR: 0.5106518, NDCG: 0.2774892, F1: 0.1227638, HIT_RATE: 0.9384536, ARHR_ALL_HITS: 0.8300048, NOVELTY: 0.0054591, AVERAGE_POPULARITY: 0.5939478, DIVERSITY_MEAN_INTER_LIST: 0.8513950, DIVERSITY_HERFINDAHL: 0.9851333, COVERAGE_ITEM: 0.0885985, COVERAGE_ITEM_CORRECT: 0.0471787, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.9372161, DIVERSITY_GINI: 0.0075744, SHANNON_ENTROPY: 7.1629117, RATIO_DIVERSITY_HERFINDAHL: 0.9855035, RATIO_DIVERSITY_GINI: 0.0292270, RATIO_SHANNON_ENTROPY: 0.5756367, RATIO_AVERAGE_POPULARITY: 2.9959664, RATIO_NOVELTY: 0.0285582, 

Iteration No: 8 ended. Evaluation done at random point.
Time taken: 21.6065
Function value obtained: -0.1360
Current minimum: -0.1380
Iteration No: 9 started. Evaluating function at random point.


SearchBayesianSkopt: New best config found. Config 14: {'alpha': 0.18201502435548725, 'beta': 0.778397170560867} - results: PRECISION: 0.2647961, PRECISION_RECALL_MIN_DEN: 0.2721457, RECALL: 0.0835725, MAP: 0.1381601, MAP_MIN_DEN: 0.1420194, MRR: 0.5234951, NDCG: 0.2820558, F1: 0.1270474, HIT_RATE: 0.9439554, ARHR_ALL_HITS: 0.8466874, NOVELTY: 0.0054456, AVERAGE_POPULARITY: 0.6015945, DIVERSITY_MEAN_INTER_LIST: 0.8438792, DIVERSITY_HERFINDAHL: 0.9843817, COVERAGE_ITEM: 0.0833933, COVERAGE_ITEM_CORRECT: 0.0443546, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.9427106, DIVERSITY_GINI: 0.0066868, SHANNON_ENTROPY: 7.0250682, RATIO_DIVERSITY_HERFINDAHL: 0.9847517, RATIO_DIVERSITY_GINI: 0.0258024, RATIO_SHANNON_ENTROPY: 0.5645591, RATIO_AVERAGE_POPULARITY: 3.0345379, RATIO_NOVELTY: 0.0284877, 

EvaluatorHoldout: Processed 13637 (100.0%) in 21.30 sec. Users per second: 640
SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'alpha': 0.18201502435548725, 'beta': 0.77839717

...

EvaluatorHoldout: Processed 13632 (100.0%) in 17.79 sec. Users per second: 766
SearchBayesianSkopt: Config 172 is suboptimal. Config: {'alpha': 0.0, 'beta': 0.010123922849606717} - results: PRECISION: 0.2640992, PRECISION_RECALL_MIN_DEN: 0.2714248, RECALL: 0.0834752, MAP: 0.1379104, MAP_MIN_DEN: 0.1417839, MRR: 0.5235664, NDCG: 0.2816149, F1: 0.1268547, HIT_RATE: 0.9433685, ARHR_ALL_HITS: 0.8460269, NOVELTY: 0.0054516, AVERAGE_POPULARITY: 0.6003012, DIVERSITY_MEAN_INTER_LIST: 0.8445554, DIVERSITY_HERFINDAHL: 0.9844493, COVERAGE_ITEM: 0.0962401, COVERAGE_ITEM_CORRECT: 0.0446315, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.9421245, DIVERSITY_GINI: 0.0070093, SHANNON_ENTROPY: 7.0470986, RATIO_DIVERSITY_HERFINDAHL: 0.9848193, RATIO_DIVERSITY_GINI: 0.0270465, RATIO_SHANNON_ENTROPY: 0.5663296, RATIO_AVERAGE_POPULARITY: 3.0280142, RATIO_NOVELTY: 0.0285194, 

Iteration No: 173 ended. Search finished for the next optimal point.
Time taken: 20.1233
Function value obtained: -0.1379
Current

Iteration No: 179 ended. Search finished for the next optimal point.
Time taken: 38.5204
Function value obtained: -0.1382
Current minimum: -0.1382
Iteration No: 180 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.6732777539940669, 'beta': 0.06438567081339976}
EvaluatorHoldout: Processed 13632 (100.0%) in 19.11 sec. Users per second: 713
SearchBayesianSkopt: Config 179 is suboptimal. Config: {'alpha': 0.6732777539940669, 'beta': 0.06438567081339976} - results: PRECISION: 0.2621112, PRECISION_RECALL_MIN_DEN: 0.2672850, RECALL: 0.0780778, MAP: 0.1340950, MAP_MIN_DEN: 0.1365908, MRR: 0.5026744, NDCG: 0.2741928, F1: 0.1203158, HIT_RATE: 0.9355927, ARHR_ALL_HITS: 0.8177245, NOVELTY: 0.0054784, AVERAGE_POPULARITY: 0.5848234, DIVERSITY_MEAN_INTER_LIST: 0.8603208, DIVERSITY_HERFINDAHL: 0.9860258, COVERAGE_ITEM: 0.0960740, COVERAGE_ITEM_CORRECT: 0.0519962, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.9343590, DIVERSITY_GINI: 0.0085661, SHANNO

Iteration No: 185 ended. Search finished for the next optimal point.
Time taken: 20.9619
Function value obtained: -0.1382
Current minimum: -0.1382
Iteration No: 186 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.5808908382337322, 'beta': 0.21413078291974125}
EvaluatorHoldout: Processed 13632 (100.0%) in 18.31 sec. Users per second: 745
SearchBayesianSkopt: Config 185 is suboptimal. Config: {'alpha': 0.5808908382337322, 'beta': 0.21413078291974125} - results: PRECISION: 0.2645833, PRECISION_RECALL_MIN_DEN: 0.2707845, RECALL: 0.0813130, MAP: 0.1370834, MAP_MIN_DEN: 0.1401914, MRR: 0.5168212, NDCG: 0.2796376, F1: 0.1243960, HIT_RATE: 0.9407277, ARHR_ALL_HITS: 0.8382044, NOVELTY: 0.0054494, AVERAGE_POPULARITY: 0.5984103, DIVERSITY_MEAN_INTER_LIST: 0.8470377, DIVERSITY_HERFINDAHL: 0.9846976, COVERAGE_ITEM: 0.0829503, COVERAGE_ITEM_CORRECT: 0.0449637, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.9394872, DIVERSITY_GINI: 0.0070765, SHANNO

Iteration No: 192 ended. Search finished for the next optimal point.
Time taken: 21.0771
Function value obtained: -0.1382
Current minimum: -0.1382
Iteration No: 193 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.2834681151138033, 'beta': 0.6016919902861505}
EvaluatorHoldout: Processed 13632 (100.0%) in 18.66 sec. Users per second: 731
SearchBayesianSkopt: Config 192 is suboptimal. Config: {'alpha': 0.2834681151138033, 'beta': 0.6016919902861505} - results: PRECISION: 0.2648107, PRECISION_RECALL_MIN_DEN: 0.2718518, RECALL: 0.0831120, MAP: 0.1381469, MAP_MIN_DEN: 0.1418444, MRR: 0.5228452, NDCG: 0.2818075, F1: 0.1265164, HIT_RATE: 0.9430018, ARHR_ALL_HITS: 0.8462156, NOVELTY: 0.0054435, AVERAGE_POPULARITY: 0.6019031, DIVERSITY_MEAN_INTER_LIST: 0.8436357, DIVERSITY_HERFINDAHL: 0.9843574, COVERAGE_ITEM: 0.0795171, COVERAGE_ITEM_CORRECT: 0.0438562, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.9417582, DIVERSITY_GINI: 0.0066083, SHANNON_

Iteration No: 199 ended. Search finished for the next optimal point.
Time taken: 21.3519
Function value obtained: -0.1381
Current minimum: -0.1382
Iteration No: 200 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.39812377366199614, 'beta': 0.5360774385535529}
EvaluatorHoldout: Processed 13632 (100.0%) in 18.96 sec. Users per second: 719
SearchBayesianSkopt: Config 199 is suboptimal. Config: {'alpha': 0.39812377366199614, 'beta': 0.5360774385535529} - results: PRECISION: 0.2648474, PRECISION_RECALL_MIN_DEN: 0.2718785, RECALL: 0.0829737, MAP: 0.1380049, MAP_MIN_DEN: 0.1416048, MRR: 0.5215791, NDCG: 0.2815218, F1: 0.1263601, HIT_RATE: 0.9424883, ARHR_ALL_HITS: 0.8449621, NOVELTY: 0.0054431, AVERAGE_POPULARITY: 0.6017748, DIVERSITY_MEAN_INTER_LIST: 0.8437072, DIVERSITY_HERFINDAHL: 0.9843645, COVERAGE_ITEM: 0.0793510, COVERAGE_ITEM_CORRECT: 0.0436901, COVERAGE_USER: 0.9986813, COVERAGE_USER_CORRECT: 0.9412454, DIVERSITY_GINI: 0.0066350, SHANNO

In [11]:
recommender = ItemKNNScoresHybridRecommender(URM_train = ICM_combined,
                                             Recommender_1=SLIME_recommender,
                                             Recommender_2=UserCF_recommender)
recommender.fit(alpha= 0.12196408034778897, beta= 0.3675922283126996 )

In [12]:
from Utils.utils import create_submission, write_submission

submission = create_submission(recommender)
write_submission(submission,"SLIME_USERCF_stacked")